In [ ]:
import sys
sys.path.append('/workspace/atma_22_ca')

import os
import numpy as np
import pandas as pd
from datetime import datetime

from configs.config import *
from src.model_arcface import ModelArcFace
from src.runner import Runner
from src.util import Logger, Validation
from src import post_processing

# ロガー初期化
run_name = f"demo_postprocessing"
logger = Logger(run_name)

## 1. データ読み込みとRunner準備

In [ ]:
# 訓練データ読み込み
df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_2nd_meta/train_meta.csv'))
logger.info(f"訓練データ: {len(df_train)}行")

# テストデータ読み込み
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_2nd_meta/test_meta.csv'))
logger.info(f"テストデータ: {len(df_test)}行")

df_test.rename(columns={'session_no': 'session'}, inplace=True)
df_test.rename(columns={'frame_in_session': 'frame'}, inplace=True)

# CV設定（StratifiedGroupKFold）
validator_normal = Validation.create_validator(
    method='stratified_group',
    n_splits=3,
    shuffle=True,
    random_state=42
)
cv_setting = {
    'validator': validator_normal,
    'group_col': 'quarter',  # グループ化列
}

# モデルパラメータ
params = {
    'model_name': 'efficientnet_b0',
    'embedding_dim': 512,
    'arcface_s': 30.0,
    'arcface_m': 0.5,
    'lr': 3e-4,
    'weight_decay': 1e-4,
    'epochs': 10,
    'batch_size': 64,
    'num_workers': 4,
    'img_size': 128,
    'threshold': 0.3,
    'use_ema': True,
    'ema_decay': 0.995,
}

logger.info("パラメータ設定完了")

## 2. Runner初期化（学習済みモデルを使用する場合）

既に学習済みのモデルがある場合はこちらを使用

In [ ]:
# 既存の学習済みモデルのrun_nameを指定
existing_run_name = "arcface_efficientnet_b0_sgkfold_202512200556"  # 実際の学習済みモデル名に変更

# Runner初期化（学習はスキップ）
runner = Runner(
    run_name=existing_run_name,
    model_cls=ModelArcFace,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger,
    seed=42
)

logger.info(f"学習済みモデル {existing_run_name} を使用")

## 3. Unknown閾値の最適化（オプション）

検証データを使ってF1スコアが最大となるUnknown閾値を探索

In [ ]:
# # Unknown閾値の最適化
# best_threshold = runner.optimize_unknown_threshold(
#     threshold_range=(0.01, 0.5),
#     num_trials=20
# )

# logger.info(f"最適なUnknown閾値: {best_threshold:.3f}")

## 4. 後処理を適用した予測

個数制約付き最適化を適用して提出ファイルを生成

In [ ]:
scores_opt = runner.metric_cv_with_postprocessing(
    unknown_prob=0.1,
    max_labels=10
)

In [ ]:
# 方法1: Unknown閾値なし（予測確率をそのまま使用）
submission_no_threshold = runner.predict_cv_with_postprocessing(
    unknown_prob=None,
    max_labels=10,
)

logger.info("予測完了（閾値なし）")
logger.info(f"Unknown(-1)の数: {(submission_no_threshold['label_id'] == -1).sum()}")

In [ ]:
import matplotlib.pyplot as plt
from src.util import Submission
pred_test = submission_no_threshold

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(10, 5))
pred_test['label_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Test Prediction Distribution (ArcFace)')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.axhline(y=len(pred_test)/12, color='r', linestyle='--', alpha=0.5, label='Uniform')
plt.legend()    
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
save_path = Submission.save(pred_test, run_name=f'{run_name}_demo', logger=logger)

In [ ]:
pd.read_csv(save_path)

## 5. 予測結果の比較

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 閾値なし
counts1 = submission_no_threshold['label_id'].value_counts().sort_index()
axes[0].bar(counts1.index, counts1.values)
axes[0].set_title('Without Threshold', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Label ID')
axes[0].set_ylabel('Count')
axes[0].grid(True, alpha=0.3)

# 閾値あり
counts2 = submission_with_threshold['label_id'].value_counts().sort_index()
axes[1].bar(counts2.index, counts2.values, color='orange')
axes[1].set_title('With Optimized Threshold', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Label ID')
axes[1].set_ylabel('Count')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/workspace/atma_22_ca/data/figures/postprocessing_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

logger.info("比較グラフを保存: data/figures/postprocessing_comparison.png")

## 6. 低レベルAPIの使用例（直接post_processingモジュールを使う）

In [ ]:
def naive_assignment(probs):
    """
    単純なargmaxによる予測割り当てを行う関数。
    
    Args:
        probs (np.ndarray): 各サンプルのクラス確率を含む2D配列（サンプル数 x クラス数）。
        
    Returns:
        np.ndarray: 各サンプルに対する予測クラスラベルの1D配列。
    """
    return np.argmax(probs, axis=2)

In [ ]:
# ダミーデータで動作確認
np.random.seed(42)

# (seq_len=10, num_samples=5, num_labels=12) の確率配列
dummy_probs = np.random.rand(10, 5, 12)
dummy_probs = dummy_probs / dummy_probs.sum(axis=2, keepdims=True)

print("ダミーデータ形状:", dummy_probs.shape)

# Naive予測（単純argmax）
naive_pred = naive_assignment(dummy_probs)
print("\nNaive予測:")
print(naive_pred)

# 最適化予測
optimized_pred = post_processing.optimize_assignment(
    dummy_probs,
    unknown_label_idx=11,
    max_labels=10,
    unknown_prob=0.1,
    verbose=True
)
print("\n最適化後:")
print(optimized_pred)

# 制約検証

# スコア比較
# naive_score = post_processing.calculate_assignment_score(naive_pred, dummy_probs)
# optimized_score = post_processing.calculate_assignment_score(optimized_pred, dummy_probs)
# print(f"\nNaiveスコア: {naive_score:.4f}")
# print(f"最適化スコア: {optimized_score:.4f}")
# print(f"改善: +{(optimized_score - naive_score):.4f} ({(optimized_score/naive_score - 1)*100:.2f}%)")

## まとめ

### 実装した機能
1. ✓ `post_processing.optimize_assignment()`: 個数制約付き最大化
2. ✓ `runner.predict_cv_with_postprocessing()`: CVモデルアンサンブル + 最適化
3. ✓ `runner.optimize_unknown_threshold()`: Unknown閾値の最適化
4. ✓ `model.predict_proba()`: 確率予測

### 期待される効果
- Discussion記事: **0.8570 → 0.9131** (+0.0561)
- さらに制約追加で: **0.9485** (+0.0915)

### 次のステップ
1. 実際の学習済みモデルで効果を検証
2. 追加の制約を実装（時系列制約など）
3. LBスコアで確認